In [1]:
pip install numpy torch torchvision torchmetrics torchprofile torchaudio scikit-learn pandas matplotlib seaborn scipy thop

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 11.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 46.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 74.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 34.2 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 30.2 MB/s  0:00:01m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 24.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 18.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17/17 [torchprofile] [torchmetrics]
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
import time
from sklearn.preprocessing import StandardScaler
from collections import Counter
from thop import profile
from IPython.display import display

def get_rotation_matrix_3d(angle_degrees, axis='z'):
    angle_rad = np.radians(angle_degrees)
    cos_a = np.cos(angle_rad)
    sin_a = np.sin(angle_rad)
    
    if axis == 'z':
        return np.array([
            [cos_a, -sin_a, 0],
            [sin_a, cos_a, 0],
            [0, 0, 1]
        ])
    elif axis == 'y':
        return np.array([
            [cos_a, 0, sin_a],
            [0, 1, 0],
            [-sin_a, 0, cos_a]
        ])
    elif axis == 'x':
        return np.array([
            [1, 0, 0],
            [0, cos_a, -sin_a],
            [0, sin_a, cos_a]
        ])

def rotate_signal(x, angle, dataset_type='uci'):
    x_rotated = x.copy()
    rot_mat = get_rotation_matrix_3d(angle, axis='z')
    
    if dataset_type == 'uci':
        for t in range(x.shape[1]):
            x_rotated[0:3, t] = rot_mat @ x[0:3, t]
            x_rotated[3:6, t] = rot_mat @ x[3:6, t]
            x_rotated[6:9, t] = rot_mat @ x[6:9, t]
    
    elif dataset_type == 'wisdm':
        for t in range(x.shape[1]):
            x_rotated[:, t] = rot_mat @ x[:, t]
    
    elif dataset_type == 'pamap2':
        for t in range(x.shape[1]):
            x_rotated[1:4, t] = rot_mat @ x[1:4, t]
            x_rotated[4:7, t] = rot_mat @ x[4:7, t]
            x_rotated[7:10, t] = rot_mat @ x[7:10, t]
            x_rotated[10:13, t] = rot_mat @ x[10:13, t]
            
            x_rotated[17:20, t] = rot_mat @ x[17:20, t]
            x_rotated[20:23, t] = rot_mat @ x[20:23, t]
            x_rotated[23:26, t] = rot_mat @ x[23:26, t]
            x_rotated[26:29, t] = rot_mat @ x[26:29, t]
            
            x_rotated[33:36, t] = rot_mat @ x[33:36, t]
            x_rotated[36:39, t] = rot_mat @ x[36:39, t]
            x_rotated[39:42, t] = rot_mat @ x[39:42, t]
            x_rotated[42:45, t] = rot_mat @ x[42:45, t]
    
    elif dataset_type == 'mhealth':
        for t in range(x.shape[1]):
            x_rotated[0:3, t] = rot_mat @ x[0:3, t]
            x_rotated[5:8, t] = rot_mat @ x[5:8, t]
            x_rotated[8:11, t] = rot_mat @ x[8:11, t]
            x_rotated[11:14, t] = rot_mat @ x[11:14, t]
            x_rotated[14:17, t] = rot_mat @ x[14:17, t]
            x_rotated[17:20, t] = rot_mat @ x[17:20, t]
            x_rotated[20:23, t] = rot_mat @ x[20:23, t]
    
    return x_rotated

class RotationTestDataset(Dataset):
    def __init__(self, X, y, rotations, dataset_type='uci'):
        self.X = X
        self.y = y
        self.rotations = rotations
        self.dataset_type = dataset_type
        
    def __len__(self):
        return len(self.X) * len(self.rotations)
    
    def __getitem__(self, idx):
        sample_idx = idx // len(self.rotations)
        rotation_idx = idx % len(self.rotations)
        
        x = self.X[sample_idx]
        y = self.y[sample_idx]
        rotation_angle = self.rotations[rotation_idx]
        
        x_rotated = rotate_signal(x, rotation_angle, self.dataset_type)
        
        return torch.FloatTensor(x_rotated), y

def split_sequences(features, labels, window_size, step):
    X_sequences, y_sequences = [], []
    for i in range(0, len(features) - window_size + 1, step):
        window_features = features[i:i + window_size]
        window_labels = labels[i:i + window_size]
        most_common_label = Counter(window_labels).most_common(1)[0][0]
        X_sequences.append(window_features)
        y_sequences.append(most_common_label)
    return np.array(X_sequences), np.array(y_sequences)

def load_mhealth_data(data_path, window_size=50, step=25):
    print("Loading mHealth dataset...")
    log_files = glob.glob(os.path.join(data_path, "*.log"))
    if not log_files:
        raise FileNotFoundError(f"No log files found in {data_path}")
    
    all_data = []
    for log_file in log_files:
        try:
            data = pd.read_csv(log_file, header=None, sep='\t')
            all_data.append(data)
        except Exception as e:
            continue
    
    if not all_data:
        raise ValueError("No valid data files found")
    
    combined_data = pd.concat(all_data, ignore_index=True)
    features = combined_data.iloc[:, :-1].values
    labels = combined_data.iloc[:, -1].values.astype(int)
    
    mask = labels != 0
    features = features[mask]
    labels = labels[mask]
    labels = labels - 1
    
    X_sequences, y_sequences = split_sequences(features, labels, window_size, step)
    unique_labels = np.unique(y_sequences)
    num_classes = len(unique_labels)
    
    activity_labels = [
        "Standing still", "Sitting and relaxing", "Lying down", "Walking", "Climbing stairs", "Waist bends forward",
        "Frontal elevation of arms", "Knees bending (crouching)", "Cycling", "Jogging", "Running", "Jump front & back"
    ]
    
    X_windowed = np.transpose(X_sequences, (0, 2, 1))
    print(f"mHealth - Shape: {X_windowed.shape}, Classes: {num_classes}")
    
    return X_windowed, y_sequences, activity_labels, num_classes

def load_uci_har_raw(dataset_path):
    print(f"Loading UCI HAR data from: {dataset_path}")
    SIGNALS = ["body_acc_x", "body_acc_y", "body_acc_z", "body_gyro_x", "body_gyro_y", "body_gyro_z", "total_acc_x", "total_acc_y", "total_acc_z"]
    
    X_train_list = []
    X_test_list = []
    
    for signal in SIGNALS:
        train_file = os.path.join(dataset_path, f"{signal}_train.txt")
        test_file = os.path.join(dataset_path, f"{signal}_test.txt")
        
        try:
            train_signal = np.genfromtxt(train_file, dtype=np.float32, invalid_raise=False)
            test_signal = np.genfromtxt(test_file, dtype=np.float32, invalid_raise=False)
            
            train_signal = train_signal[~np.isnan(train_signal).any(axis=1)]
            test_signal = test_signal[~np.isnan(test_signal).any(axis=1)]
            
            X_train_list.append(train_signal)
            X_test_list.append(test_signal)
        except:
            return None, None, None, None, None
    
    if not X_train_list:
        return None, None, None, None, None
    
    X_train = np.stack(X_train_list, axis=-1)
    X_test = np.stack(X_test_list, axis=-1)
    
    X_train = np.transpose(X_train, (0, 2, 1))
    X_test = np.transpose(X_test, (0, 2, 1))
    
    try:
        y_train = np.genfromtxt(os.path.join(dataset_path, "y_train.txt"), dtype=int, invalid_raise=False) - 1
        y_test = np.genfromtxt(os.path.join(dataset_path, "y_test.txt"), dtype=int, invalid_raise=False) - 1
        
        y_train = y_train[~np.isnan(y_train)].astype(int)
        y_test = y_test[~np.isnan(y_test)].astype(int)
    except:
        return None, None, None, None, None
    
    ucihar_activity_names = ['Walking', 'Walking Upstairs', 'Walking Downstairs', 'Sitting', 'Standing', 'Laying']
    print(f"UCI HAR - Shape: {X_train.shape}, Classes: {len(ucihar_activity_names)}")
    
    return X_train, y_train, X_test, y_test, ucihar_activity_names

def load_wisdm_data(data_path, window_size=80, step=40):
    print("Loading WISDM dataset...")
    
    if os.path.isdir(data_path):
        possible_files = ['WISDM_ar_v1.1_raw.txt', 'WISDM_ar_v1.1.txt']
        for file in possible_files:
            file_path = os.path.join(data_path, file)
            if os.path.exists(file_path):
                data_path = file_path
                break
    
    column_names = ['user', 'activity', 'timestamp', 'x_accel', 'y_accel', 'z_accel']
    rows = []
    
    with open(data_path, 'r') as file:
        for line in file:
            line = line.strip()
            if line.endswith(';'):
                line = line[:-1]
            if line:
                try:
                    values = line.split(',')
                    if len(values) == 6:
                        rows.append(values)
                except:
                    continue
    
    df = pd.DataFrame(rows, columns=column_names)
    
    for col in ['user', 'timestamp', 'x_accel', 'y_accel', 'z_accel']:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    df = df.dropna()
    
    label_encoder = LabelEncoder()
    encoded_labels = label_encoder.fit_transform(df['activity'])
    
    features = df[['x_accel', 'y_accel', 'z_accel']].values
    X_sequences, y_sequences = split_sequences(features, encoded_labels, window_size, step)
    
    activity_labels = label_encoder.classes_.tolist()
    num_classes = len(activity_labels)
    
    X_windowed = np.transpose(X_sequences, (0, 2, 1))
    print(f"WISDM - Shape: {X_windowed.shape}, Classes: {num_classes}")
    
    return X_windowed, y_sequences, activity_labels, num_classes

def load_and_window_pamap2(dataset_dir="PAMAP2"):
    file_paths = sorted(glob.glob(os.path.join(dataset_dir, 'Protocol', 'subject*.dat')))
    optional_path = os.path.join(dataset_dir, 'Optional')
    if os.path.exists(optional_path):
        file_paths += sorted(glob.glob(os.path.join(optional_path, 'subject*.dat')))
    if not file_paths:
        return None, None, None, None

    activity_labels = [
        "lying", "sitting", "standing", "walking", "running", "cycling",
        "Nordic walking", "ascending stairs", "descending stairs",
        "vacuum cleaning", "ironing", "rope jumping"
    ]
    
    label_to_activity_idx = {
        1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 12: 7, 13: 8, 16: 9, 17: 10, 24: 11
    }

    all_windows = []
    all_labels = []
    window_size = 100
    step = 50

    for file_path in file_paths:
        df = pd.read_csv(file_path, sep=r'\s+', header=None, na_values='NaN')
        df_cleaned = df.ffill().bfill()
        if df_cleaned.empty:
            continue
        
        labels = df_cleaned.iloc[:, 1].values.astype(int)
        
        hand_cols = list(range(4, 20))
        chest_cols = list(range(21, 37))
        ankle_cols = list(range(38, 54))
        all_sensor_cols = hand_cols + chest_cols + ankle_cols

        if df_cleaned.shape[1] < max(all_sensor_cols) + 1:
            continue
        features = df_cleaned.iloc[:, all_sensor_cols].values.astype(np.float32)

        valid_indices = np.where(np.isin(labels, list(label_to_activity_idx.keys())))[0]
        if len(valid_indices) == 0:
            continue
        
        features = features[valid_indices, :]
        labels = labels[valid_indices]
        
        if len(features) < window_size:
            continue
        
        start = 0
        while start + window_size <= len(features):
            window_data = features[start : start + window_size, :]
            window_labels_raw = labels[start : start + window_size]
            most_common_label = Counter(window_labels_raw).most_common(1)[0][0]
            
            if most_common_label in label_to_activity_idx:
                all_windows.append(window_data)
                all_labels.append(label_to_activity_idx[most_common_label])
            start += step

    if not all_windows:
        return None, None, None, None

    X_windowed = np.array(all_windows, dtype=np.float32)
    y_encoded = np.array(all_labels, dtype=int)

    scaler = StandardScaler()
    X_windowed_flat = X_windowed.reshape(X_windowed.shape[0], -1)
    X_windowed_flat = scaler.fit_transform(X_windowed_flat)
    X_windowed = X_windowed_flat.reshape(X_windowed.shape)
    X_windowed = np.transpose(X_windowed, (0, 2, 1))

    num_classes_actual = len(np.unique(y_encoded))
    
    print("\n" + "="*40)
    print("PAMAP2 Dataset Loading Summary")
    print(f"  - Total Instances (Windows): {X_windowed.shape[0]}")
    print(f"  - Total Features: {X_windowed.shape[1]}")
    print(f"  - Total Classes: {num_classes_actual}")
    print("="*40 + "\n")
    
    return X_windowed, y_encoded, activity_labels, num_classes_actual

class DSConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0):
        super(DSConv, self).__init__()
        self.depthwise = nn.Conv1d(in_channels, in_channels, kernel_size, stride, padding, groups=in_channels)
        self.pointwise = nn.Conv1d(in_channels, out_channels, 1)
        
    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        return x

class InceptionBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(InceptionBlock, self).__init__()
        
        self.branch1 = nn.Sequential(
            nn.MaxPool1d(3, stride=1, padding=1),
            DSConv(in_channels, out_channels//4, 1)
        )
        
        self.branch2 = nn.Sequential(
            DSConv(in_channels, out_channels//4, 1),
            DSConv(out_channels//4, out_channels//4, 1)
        )
        
        self.branch3 = nn.Sequential(
            DSConv(in_channels, out_channels//4, 1),
            DSConv(out_channels//4, out_channels//4, 1)
        )
        
        self.branch4 = DSConv(in_channels, out_channels//4, 1)
        
        self.relu = nn.ReLU()
        
    def forward(self, x):
        b1 = self.branch1(x)
        b2 = self.branch2(x)
        b3 = self.branch3(x)
        b4 = self.branch4(x)
        
        out = torch.cat([b1, b2, b3, b4], dim=1)
        out = self.relu(out)
        return out

class MultiKernelBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(MultiKernelBlock, self).__init__()
        
        self.branch1 = DSConv(in_channels, out_channels//4, 1)
        self.branch2 = DSConv(in_channels, out_channels//4, 3, padding=1)
        self.branch3 = DSConv(in_channels, out_channels//4, 5, padding=2)
        self.branch4 = DSConv(in_channels, out_channels//4, 7, padding=3)
        
        self.relu = nn.ReLU()
        
    def forward(self, x):
        b1 = self.branch1(x)
        b2 = self.branch2(x)
        b3 = self.branch3(x)
        b4 = self.branch4(x)
        
        out = torch.cat([b1, b2, b3, b4], dim=1)
        out = self.relu(out)
        return out

class StemLayer(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(StemLayer, self).__init__()
        self.conv1d = nn.Conv1d(in_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.conv1d(x)
        x = self.bn(x)
        x = self.relu(x)
        return x

class HARModel(nn.Module):
    def __init__(self, in_channels, num_classes, stem_out=64, inception_out=128, mk_out=128):
        super(HARModel, self).__init__()
        
        self.stem = StemLayer(in_channels, stem_out)
        self.inception = InceptionBlock(stem_out, inception_out)
        self.mk = MultiKernelBlock(inception_out, mk_out)
        
        self.adaptive_pool = nn.AdaptiveAvgPool1d(1)
        self.flatten = nn.Flatten()
        
        self.fc1 = nn.Linear(mk_out, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.3)
        
        self.fc2 = nn.Linear(256, num_classes)
        
    def forward(self, x):
        x = self.stem(x)
        x = self.inception(x)
        x = self.mk(x)
        x = self.adaptive_pool(x)
        x = self.flatten(x)
        
        features = x
        
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        out = self.fc2(x)
        
        return out, features

def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, device, epochs=15):
    start_time = time.time()
    
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs, _ = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
        
        train_acc = 100. * correct / total
    
        
        scheduler.step()
    
    training_time = time.time() - start_time
    return training_time

def evaluate_model(model, test_loader, device):
    model.eval()
    
    all_preds = []
    all_labels = []
    all_features = []
    
    inference_times = []
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            
            start = time.time()
            outputs, features = model(inputs)
            inference_times.append((time.time() - start) * 1000)
            
            _, predicted = outputs.max(1)
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.numpy())
            all_features.append(features.cpu().numpy())
    
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    
    if all_features:
        all_features = np.vstack(all_features)
    else:
        all_features = np.array([])
    
    acc = accuracy_score(all_labels, all_preds) * 100
    f1 = f1_score(all_labels, all_preds, average='weighted') * 100
    precision = precision_score(all_labels, all_preds, average='weighted', zero_division=0) * 100
    recall = recall_score(all_labels, all_preds, average='weighted', zero_division=0) * 100
    avg_inference_time = np.mean(inference_times) if inference_times else 0
    
    return acc, f1, precision, recall, all_preds, all_labels, all_features, avg_inference_time

def compute_model_stats(model, input_shape):
    device = next(model.parameters()).device
    dummy_input = torch.randn(1, *input_shape).to(device)
    
    macs, params = profile(model, inputs=(dummy_input,), verbose=False)
    
    total_macs = macs / 1e6
    total_params = params / 1e6
    
    return total_params, total_macs

def plot_confusion_matrix(y_true, y_pred, activity_names, dataset_name, suffix=""):
    cm = confusion_matrix(y_true, y_pred)
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    df = pd.DataFrame(cm_normalized, index=activity_names, columns=activity_names)
    
    annot = df.copy().astype(str)
    for i in range(df.shape[0]):
        for j in range(df.shape[1]):
            v = df.iloc[i, j]
            annot.iloc[i, j] = f"{v:.2f}"
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(df, annot=annot.values, fmt="", cmap="Blues", cbar=True, annot_kws={"size": 13}, vmin=0, vmax=1)
    plt.xticks(rotation=90, fontsize=13)
    plt.yticks(rotation=0, fontsize=13)
    
    for spine in plt.gca().spines.values():
        spine.set_visible(True)
        spine.set_linewidth(0.5)
        spine.set_edgecolor('black')
    
    plt.xlabel('Predicted Label', fontsize=14)
    plt.ylabel('True Label', fontsize=14)
    plt.tight_layout()
    plt.savefig(f'./{dataset_name}_confusion{suffix}.png', dpi=300, bbox_inches='tight')
    plt.close()

def plot_tsne(features, labels, activity_names, dataset_name="Dataset", samples_per_class=250):
    if len(features) == 0 or np.any(np.isnan(features)) or np.any(np.isinf(features)):
        print(f"Warning: Invalid features for t-SNE in {dataset_name}. Skipping...")
        return
    
    sampled_features, sampled_labels = [], []
    for i in range(len(activity_names)):
        class_mask = labels == i
        class_features = features[class_mask]
        if len(class_features) > samples_per_class:
            idx = np.random.choice(len(class_features), samples_per_class, replace=False)
            sampled_features.append(class_features[idx])
            sampled_labels.append(labels[class_mask][idx])
        else:
            if len(class_features) > 0:
                sampled_features.append(class_features)
                sampled_labels.append(labels[class_mask])
    
    if not sampled_features:
        print(f"Warning: No valid samples for t-SNE in {dataset_name}. Skipping...")
        return
    
    features = np.vstack(sampled_features)
    labels = np.hstack(sampled_labels)
    
    features_2d = TSNE(n_components=2, perplexity=20, learning_rate=3000).fit_transform(features)
    
    colors = [
        "#FF0000", "#FFA500", "#FFFF00", "#008000", "#00FFFF", "#0000FF",
        "#800080", "#FFC0CB", "#A52A2A", "#000000", "#808080", "#FFD700"
    ]
    
    plt.figure(figsize=(8, 6))
    for i, activity in enumerate(activity_names):
        mask = labels == i
        if np.any(mask):
            plt.scatter(
                features_2d[mask, 0], features_2d[mask, 1],
                color=colors[i], marker='o', s=20, alpha=0.5
            ) 
    
    handles = [
        plt.Line2D([0], [0], marker='o', color='w', label=activity,
                   markerfacecolor=colors[i], markersize=7)
        for i, activity in enumerate(activity_names)
    ]
    plt.legend(handles=handles, title="Activities", fontsize=9, framealpha=1)
    plt.xlabel("t-SNE Component 1", fontsize=13)
    plt.ylabel("t-SNE Component 2", fontsize=13)
    plt.grid(False)
    plt.savefig(f'./{dataset_name}.png', dpi=500)
    plt.close()

def run_experiment(X_data, y_data, activity_names, dataset_name, dataset_type, train_ratio, device):
    print(f"\n{dataset_name} - {int(train_ratio*100)}% labeled data")
    
    n_samples = len(X_data)
    
    if train_ratio == 1.0:
        train_size = 0.8
        val_size = 0.1
        
        X_temp, X_test, y_temp, y_test = train_test_split(
            X_data, y_data, test_size=1-train_size, stratify=y_data, random_state=42
        )
        
        X_train, X_val, y_train, y_val = train_test_split(
            X_temp, y_temp, test_size=val_size/(train_size), stratify=y_temp, random_state=42
        )
    else:
        X_labeled, X_test, y_labeled, y_test = train_test_split(
            X_data, y_data, test_size=1-train_ratio, stratify=y_data, random_state=42
        )
        
        X_train, X_val, y_train, y_val = train_test_split(
            X_labeled, y_labeled, test_size=0.2, stratify=y_labeled, random_state=42
        )
    
    in_channels = X_data.shape[1]
    num_classes = len(activity_names)
    
    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.LongTensor(y_train))
    val_dataset = TensorDataset(torch.FloatTensor(X_val), torch.LongTensor(y_val))
    test_dataset = TensorDataset(torch.FloatTensor(X_test), torch.LongTensor(y_test))
    
    train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=512, shuffle=False, num_workers=0)
    test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False, num_workers=0)
    
    model = HARModel(in_channels, num_classes).to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)
    
    train_time = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, device, epochs=35)
    
    acc1, f1_1, precision1, recall1, all_preds1, all_labels1, all_features1, inference_time1 = evaluate_model(model, test_loader, device)
    
    params, flops = compute_model_stats(model, (in_channels, X_data.shape[2]))
    
    print(f"Test 1 (Original): Acc={acc1:.2f}%, F1={f1_1:.2f}%")
    
    rotations = [0, 90, 180, 270]
    test2_dataset = RotationTestDataset(X_test, y_test, rotations, dataset_type)
    test2_loader = DataLoader(test2_dataset, batch_size=512, shuffle=False, num_workers=0)
    acc2, f1_2, precision2, recall2, all_preds2, all_labels2, all_features2, inference_time2 = evaluate_model(model, test2_loader, device)
    
    print(f"Test 2 (Train Rotations): Acc={acc2:.2f}%, F1={f1_2:.2f}%")
    
    small_rotations = [0, 5, 10, 15, 20, 25, 30]
    test3_dataset = RotationTestDataset(X_test, y_test, small_rotations, dataset_type)
    test3_loader = DataLoader(test3_dataset, batch_size=512, shuffle=False, num_workers=0)
    acc3, f1_3, precision3, recall3, all_preds3, all_labels3, all_features3, inference_time3 = evaluate_model(model, test3_loader, device)
    
    print(f"Test 3 (Small Angles): Acc={acc3:.2f}%, F1={f1_3:.2f}%")
    
    if train_ratio == 1.0:
        plot_confusion_matrix(all_labels1, all_preds1, activity_names, f"{dataset_name}_{int(train_ratio*100)}pct", "")
        
        model.eval()
        train_features = []
        train_labels = []
        with torch.no_grad():
            for inputs, labels in train_loader:
                inputs = inputs.to(device)
                _, features = model(inputs)
                train_features.append(features.cpu().numpy())
                train_labels.extend(labels.numpy())
        
        if train_features:
            train_features = np.vstack(train_features)
            train_labels = np.array(train_labels)
            plot_tsne(train_features, train_labels, activity_names, f"{dataset_name}_{int(train_ratio*100)}pct_tsne")
        
        torch.save(model.state_dict(), f'./{dataset_name}_{int(train_ratio*100)}pct_model.pth')
    
    result = {
        'Dataset': dataset_name,
        'Train%': int(train_ratio*100),
        'Test1_Acc': acc1,
        'Test1_F1': f1_1,
        'Test1_Precision': precision1,
        'Test1_Recall': recall1,
        'Test2_Acc': acc2,
        'Test2_F1': f1_2,
        'Test2_Precision': precision2,
        'Test2_Recall': recall2,
        'Test3_Acc': acc3,
        'Test3_F1': f1_3,
        'Test3_Precision': precision3,
        'Test3_Recall': recall3,
        'Params(M)': params,
        'FLOPs(M)': flops,
        'InferenceTime(ms)': inference_time1,
        'TrainTime(s)': train_time
    }
    
    return result

print("="*80)
print("LOADING DATASETS")
print("="*80)

uci_X_train, uci_y_train, uci_X_test, uci_y_test, uci_activity_names = load_uci_har_raw('./')
uci_X = np.concatenate([uci_X_train, uci_X_test], axis=0)
uci_y = np.concatenate([uci_y_train, uci_y_test], axis=0)

wisdm_X, wisdm_y, wisdm_activity_names, wisdm_num_classes = load_wisdm_data('./WISDM')

pamap2_X, pamap2_y, pamap2_activity_names, pamap2_num_classes = load_and_window_pamap2('./PAMAP2')

mhealth_X, mhealth_y, mhealth_activity_names, mhealth_num_classes = load_mhealth_data('./MHEALTH')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}\n")

all_results = []

datasets = [
    (uci_X, uci_y, uci_activity_names, 'UCI', 'uci'),
    (wisdm_X, wisdm_y, wisdm_activity_names, 'WISDM', 'wisdm'),
    (pamap2_X, pamap2_y, pamap2_activity_names, 'PAMAP2', 'pamap2'),
    (mhealth_X, mhealth_y, mhealth_activity_names, 'mHealth', 'mhealth')
]

train_ratios = [0.01, 0.05, 0.1, 0.5, 1.0]

print("="*80)
print("SUPERVISED TRAINING")
print("="*80)

for X_data, y_data, activity_names, dataset_name, dataset_type in datasets:
    for train_ratio in train_ratios:
        result = run_experiment(X_data, y_data, activity_names, dataset_name, dataset_type, train_ratio, device)
        
        all_results.append(result)
        
        result_df = pd.DataFrame([result])
        display(result_df)

print("\n" + "="*80)
print("FINAL RESULTS")
print("="*80)

rst_df = pd.DataFrame(all_results)
display(rst_df)

rst_df.to_csv('./supervised_results.csv', index=False)
print("\nResults saved to './supervised_results.csv'")

LOADING DATASETS
Loading UCI HAR data from: ./
UCI HAR - Shape: (7352, 9, 128), Classes: 6
Loading WISDM dataset...
WISDM - Shape: (27160, 3, 80), Classes: 6

PAMAP2 Dataset Loading Summary
  - Total Instances (Windows): 23822
  - Total Features: 48
  - Total Classes: 12

Loading mHealth dataset...


/tmp/ipykernel_500/1282408614.py:139: RuntimeWarning: invalid value encountered in cast
  labels = combined_data.iloc[:, -1].values.astype(int)


mHealth - Shape: (12348, 23, 50), Classes: 12
Using device: cuda

SUPERVISED TRAINING

UCI - 1% labeled data
Test 1 (Original): Acc=26.55%, F1=13.98%
Test 2 (Train Rotations): Acc=24.62%, F1=12.95%
Test 3 (Small Angles): Acc=22.75%, F1=11.34%


,Dataset,Train%,Test1_Acc,Test1_F1,Test1_Precision,Test1_Recall,Test2_Acc,Test2_F1,Test2_Precision,Test2_Recall,Test3_Acc,Test3_F1,Test3_Precision,Test3_Recall,Params(M),FLOPs(M),InferenceTime(ms),TrainTime(s)
0,UCI,1,26.547024,13.98189,10.400315,26.547024,24.62489,12.945431,19.725906,24.62489,22.746186,11.337835,10.050181,22.746186,0.067142,4.016768,1.336706,0.386345



UCI - 5% labeled data
Test 1 (Original): Acc=41.67%, F1=34.61%
Test 2 (Train Rotations): Acc=27.35%, F1=20.75%
Test 3 (Small Angles): Acc=37.69%, F1=29.44%


...


mHealth - 10% labeled data
Test 1 (Original): Acc=32.80%, F1=20.04%
Test 2 (Train Rotations): Acc=24.16%, F1=14.08%
Test 3 (Small Angles): Acc=32.01%, F1=19.95%


,Dataset,Train%,Test1_Acc,Test1_F1,Test1_Precision,Test1_Recall,Test2_Acc,Test2_F1,Test2_Precision,Test2_Recall,Test3_Acc,Test3_F1,Test3_Precision,Test3_Recall,Params(M),FLOPs(M),InferenceTime(ms),TrainTime(s)
0,mHealth,10,32.796473,20.042345,29.506359,32.796473,24.160968,14.078299,27.885894,24.160968,32.00982,19.946364,30.795797,32.00982,0.071372,1.726592,1.182784,1.096517



mHealth - 50% labeled data
Test 1 (Original): Acc=9.23%, F1=1.56%
Test 2 (Train Rotations): Acc=9.23%, F1=1.56%
Test 3 (Small Angles): Acc=9.23%, F1=1.56%


,Dataset,Train%,Test1_Acc,Test1_F1,Test1_Precision,Test1_Recall,Test2_Acc,Test2_F1,Test2_Precision,Test2_Recall,Test3_Acc,Test3_F1,Test3_Precision,Test3_Recall,Params(M),FLOPs(M),InferenceTime(ms),TrainTime(s)
0,mHealth,50,9.232264,1.560614,0.852347,9.232264,9.232264,1.560614,0.852347,9.232264,9.232264,1.560614,0.852347,9.232264,0.071372,1.726592,1.261895,5.764637



mHealth - 100% labeled data


In [36]:
rm -rf ./MHEALTH ./PAMAP2 ./WISDM